In [133]:

from redis import Redis
from typing import List, Set
from jamboree.utils.helper import Helpers
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle

## Summary -

In this notebook, I experimented saving PyTorch Nueral Network Models into Redis using ZADD in order to maintain version control. In order to maintain model version control, I used the version number to score each model that is entered into the Redis. Therefore, you can quickly obtain either the latest version, or any specific version of a Model.

In [134]:
redis = Redis()
helper = Helpers()

In [135]:
#Basic example Nueral Net using Pytorch -> https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    


In [136]:
def randomInput(model):
    randInput = torch.randn(1, 1, 32, 32)
    result = model(randInput)
    return(result)

In [137]:
model = Net()
print(randomInput(model))


tensor([[ 0.1517, -0.0148,  0.0918, -0.0704,  0.0521,  0.0663, -0.1232,  0.0533,
          0.1535,  0.0860]], grad_fn=<AddmmBackward>)


In [138]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [139]:
def getParams(model):
    params = []
    for param in model.parameters():
        params.append(param.data)
        
    return params

In [140]:
print(getParams(model))

[tensor([[[[ 0.1002,  0.2539, -0.3083],
          [-0.1413,  0.2881,  0.2135],
          [ 0.1940, -0.0133, -0.2575]]],


        [[[ 0.3048, -0.0649, -0.2526],
          [-0.0468,  0.2841, -0.1097],
          [-0.0182, -0.0556, -0.0317]]],


        [[[ 0.2261,  0.2988,  0.0935],
          [-0.0813, -0.0220, -0.3232],
          [-0.2744,  0.2896, -0.0365]]],


        [[[-0.2438, -0.1938, -0.2583],
          [ 0.2164,  0.2481,  0.2002],
          [-0.2293, -0.1449,  0.2018]]],


        [[[-0.1128,  0.2783, -0.1782],
          [-0.1449, -0.1703,  0.2805],
          [ 0.0732,  0.0554, -0.1508]]],


        [[[-0.2184, -0.2776,  0.3214],
          [ 0.2378,  0.3279, -0.1351],
          [-0.2381, -0.1002,  0.1011]]]]), tensor([-0.3273,  0.0882,  0.1454,  0.3116, -0.3074, -0.1088]), tensor([[[[ 0.1075, -0.0719,  0.1092],
          [-0.0592,  0.0899,  0.0546],
          [ 0.1080,  0.0898, -0.0708]],

         [[-0.1355,  0.0155, -0.0798],
          [ 0.0784, -0.0299, -0.1171],
          [ 

In [141]:
def dictify(model) -> dict:
    modelDict = {
        'model': Net(),
        'state_dict': model.state_dict(),
        'optimizer' : optimizer.state_dict()
    }
    return modelDict

In [142]:
def dedictify(modelDict:dict):
    model = modelDict['model']
    model.load_state_dict(modelDict['state_dict'])
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    optimizer.load_state_dict(modelDict['optimizer'])
    return model, optimizer



## Saving Model

In [193]:
def save(model, key:str, version):
    key = helper.generate_hash(key)
    modelDict = dictify(model)
    pickled = pickle.dumps(modelDict)
    scored = {pickled:version}
    redis.zadd(key, scored)
    print("Model Saved")

In [203]:
save(model, "model_key", 3.01)

Model Saved


## Loading Models

In [204]:
def get_latest_model(key:str):
    key = helper.generate_hash(key)
    pickled = redis.zrange(key, -1, -1)
    modelDict = pickle.loads(pickled[0])
    return(dedictify(modelDict))
    

In [214]:
def get_model_by_version(key:str, version:float):
    key = helper.generate_hash(key)
    pickled = redis.zrange(key, version, version)
    modelDict = pickle.loads(pickled[0])
    return(dedictify(modelDict))

In [215]:
def get_latest_version_number(key:str):
    key = helper.generate_hash(key)
    pickled = redis.zrange(key, -1, -1, withscores=True)
    return(pickled[0][1])

In [216]:
model, optimizer = get_latest_model('model_key')

In [217]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[[[ 0.1002,  0.2539, -0.3083],
          [-0.1413,  0.2881,  0.2135],
          [ 0.1940, -0.0133, -0.2575]]],


        [[[ 0.3048, -0.0649, -0.2526],
          [-0.0468,  0.2841, -0.1097],
          [-0.0182, -0.0556, -0.0317]]],


        [[[ 0.2261,  0.2988,  0.0935],
          [-0.0813, -0.0220, -0.3232],
          [-0.2744,  0.2896, -0.0365]]],


        [[[-0.2438, -0.1938, -0.2583],
          [ 0.2164,  0.2481,  0.2002],
          [-0.2293, -0.1449,  0.2018]]],


        [[[-0.1128,  0.2783, -0.1782],
          [-0.1449, -0.1703,  0.2805],
          [ 0.0732,  0.0554, -0.1508]]],


        [[[-0.2184, -0.2776,  0.3214],
          [ 0.2378,  0.3279, -0.1351],
          [-0.2381, -0.1002,  0.1011]]]], requires_grad=True)
Parameter containing:
tensor([-0.3273,  0.0882,  0.1454,  0.3116, -0.3074, -0.1088],
       requires_grad=True)
Parameter containing:
tensor([[[[ 0.1075, -0.0719,  0.1092],
          [-0.0592,  0.0899,  0.0546],
          [ 0.1080,  

In [219]:
model, optimizer = get_model_by_version('model_key', 3)

In [220]:
print(get_latest_version_number('model_key'))

3.01
